## 1.Dataset Load & Preprocess (DRUG_MECHANISM_FRAG Table & SUBSTANCE_ACTIVITY Table)

In [ ]:
from google.colab import drive

drive.mount('/content/Gdrive')
!cd /content/Gdrive/
!pwd

Mounted at /content/Gdrive
/content


In [ ]:
import pandas as pd

# Create a list of file names
file_names_DRUG_MECHANISM_FRAG = [f'Gdrive/MyDrive/[Your Directory]/DRUG_MECHANISM_FRAG_{i}.csv' for i in range(1, 19)]

# Create an empty list for dataframes
dataframes_DRUG_MECHANISM_FRAG = []

# Read each file and add it to the list
for file_name in file_names_DRUG_MECHANISM_FRAG:
    df = pd.read_csv(file_name)
    dataframes_DRUG_MECHANISM_FRAG.append(df)

# Merge the dataframes into one
DRUG_MECHANISM_FRAG = pd.concat(dataframes_DRUG_MECHANISM_FRAG, ignore_index=True)

# Verify the result
print(DRUG_MECHANISM_FRAG.head())
print(DRUG_MECHANISM_FRAG.shape)

   DRUG_MECHANSIM_ID  AID     REGULATION  FUNCTION_ID FUNCTION_NAME  \
0                  1    1  downregulates         1748        growth   
1                  2    3  downregulates         1748        growth   
2                  3    5  downregulates         1748        growth   
3                  4    7  downregulates         1748        growth   
4                  5    9  downregulates         1748        growth   

        FUNCTION_TYPE  GENE_ID GENE_NAME  CELL_LINE_ID CELL_LINE_NAME  \
0  biological_process      NaN       NaN           NaN            NaN   
1  biological_process      NaN       NaN           NaN            NaN   
2  biological_process      NaN       NaN           NaN            NaN   
3  biological_process      NaN       NaN           NaN            NaN   
4  biological_process      NaN       NaN           NaN            NaN   

   TISSUE_ID TISSUE_NAME  ORGANISM_ID ORGANISM_NAME  
0        NaN         NaN          NaN           NaN  
1        NaN         NaN  

In [ ]:
import pandas as pd

# Create a list of file names
file_names_SUBSTANCE_ACTIVITY = [f'Gdrive/MyDrive/[Your Directory]/SUBSTANCE_ACTIVITY_{i}.csv' for i in range(1, 4)]

# Create an empty list for dataframes
dataframes_SUBSTANCE_ACTIVITY = []

# Read each file and add it to the list
for file_name in file_names_SUBSTANCE_ACTIVITY:
    df = pd.read_csv(file_name)
    dataframes_SUBSTANCE_ACTIVITY.append(df)

# Merge the dataframes into one
SUBSTANCE_ACTIVITY = pd.concat(dataframes_SUBSTANCE_ACTIVITY, ignore_index=True)

# Verify the result
print(SUBSTANCE_ACTIVITY.head())
print(SUBSTANCE_ACTIVITY.shape)

   AID     SID  ACTIVITY
0    1   66954         1
1    1   66954         1
2    1   66970         1
3    1   66977         1
4    1  538184         1
(264013943, 3)


In [ ]:
# SID Selection: ex/ 123092557 - Pifithrin-Mu
SID = 123092557
SUBSTANCE_ACTIVITY_SID_Filtered = SUBSTANCE_ACTIVITY[SUBSTANCE_ACTIVITY['SID'] == SID]
SUBSTANCE_ACTIVITY_SID_Filtered

,AID,SID,ACTIVITY
141064311,512976,123092557,2
141064312,512977,123092557,2
141064313,512978,123092557,2
141064314,512979,123092557,2
141064315,512980,123092557,2
...,...,...,...
263884503,1889149,123092557,2
263884506,1889152,123092557,2
263884509,1889155,123092557,2
263884512,1889158,123092557,2


In [ ]:
# Extract the unique AID values from the first DataFrame
AID_Values = SUBSTANCE_ACTIVITY_SID_Filtered['AID'].unique()

# Filter the second DataFrame to keep only rows with AID values present in the first DataFrame
DRUG_MECHANISM_FRAG_Filtered_by_SID = DRUG_MECHANISM_FRAG[DRUG_MECHANISM_FRAG['AID'].isin(AID_Values)]

# Print the filtered DataFrame
print(DRUG_MECHANISM_FRAG_Filtered_by_SID)

         DRUG_MECHANSIM_ID      AID     REGULATION  FUNCTION_ID  \
830587              830588   512976  downregulates         2575   
830588              830589   512976  downregulates          486   
830589              830590   512976            NaN          363   
830590              830591   512976    upregulates         3213   
830591              830592   512976  downregulates         3137   
...                    ...      ...            ...          ...   
3183788            3183789  1889161  downregulates          977   
3183789            3183790  1889161  downregulates          977   
3183790            3183791  1889161            NaN         1039   
3183791            3183792  1889161            NaN         1039   
3183792            3183793  1889161            NaN         1039   

                                             FUNCTION_NAME  \
830587                           mitochondrion inheritance   
830588                                             binding   
830589   

## 2.Knowledge Graph Database of Drug Mechanisms(DrugMecha)

### 2.1.Neo4j Database Connection

In [ ]:
!pip install py2neo

from py2neo import Graph, Node, Relationship
import logging

# 로깅을 활성화하여 추가적인 정보를 얻습니다.
logging.basicConfig(level=logging.DEBUG)

# Try connection (Graph Database - Neo4j)
try:
    graph = Graph("neo4j+s://[Your_Domain].databases.neo4j.io", auth=("neo4j", "[your password]"))
    print("Connected to Neo4j!")
except Exception as e:
    print("Failed to connect to Neo4j:", e)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.7 MB/s eta 0:00:00
Connected to Neo4j!


### 2.2.Knowledge Graph Generation Algorithm

In [ ]:
##### Knowledge Graph Generation Algorithm: SID -> AID -> Molecular Function -> Gene -> Biological Porcess -> Cell Line -> Tissue -> Organism #####

# Delete all content from the database
graph.delete_all()

# Dictionary to store the last node for each AID
last_nodes_dict = {}

# Create and merge the SID node
sid_node = Node("SID", id=123092557) ### SID = 123092557 ###
graph.merge(sid_node, "SID", "id")

# Iterate through the dataframe to create nodes and relationships in the graph database
for index, row in DRUG_MECHANISM_FRAG_Filtered_by_SID.iterrows():
    last_node = sid_node

    # Create and link AID node with SID
    if pd.notna(row['AID']):
        aid_node = Node("AID", id=row['AID'])
        graph.merge(aid_node, "AID", "id")
        graph.create(Relationship(last_node, "regulates", aid_node))
        last_node = aid_node

    # Create and link Molecular Function node with AID
    if pd.notna(row['FUNCTION_NAME']) and row['FUNCTION_TYPE'] == "molecular_function":
        mf_node = Node("MolecularFunction", name=row['FUNCTION_NAME'], id=row['FUNCTION_ID'])
        graph.merge(mf_node, "MolecularFunction", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, mf_node))
        last_node = mf_node

    # Create and link Gene node with Molecular Function
    if pd.notna(row['GENE_NAME']):
        gene_node = Node("Gene", name=row['GENE_NAME'], id=row['GENE_ID'])
        graph.merge(gene_node, "Gene", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, gene_node))
        last_node = gene_node

    # Create and link Biological Process node with Gene
    if pd.notna(row['FUNCTION_NAME']) and row['FUNCTION_TYPE'] == "biological_process":
        bp_node = Node("BiologicalProcess", name=row['FUNCTION_NAME'], id=row['FUNCTION_ID'])
        graph.merge(bp_node, "BiologicalProcess", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, bp_node))
        last_node = bp_node

    # Create and link Cell Line node with Biological Process
    if pd.notna(row['CELL_LINE_NAME']):
        cell_line_node = Node("CellLine", name=row['CELL_LINE_NAME'])
        graph.merge(cell_line_node, "CellLine", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, cell_line_node))
        last_node = cell_line_node

    # Create and link Tissue node with Cell Line
    if pd.notna(row['TISSUE_NAME']):
        tissue_node = Node("Tissue", name=row['TISSUE_NAME'])
        graph.merge(tissue_node, "Tissue", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, tissue_node))
        last_node = tissue_node

    # Create and link Organism node with Tissue
    if pd.notna(row['ORGANISM_NAME']):
        organism_node = Node("Organism", name=row['ORGANISM_NAME'], id=row['ORGANISM_ID'])
        graph.merge(organism_node, "Organism", "name")
        if last_node:
            regulation = row['REGULATION'] if pd.notna(row['REGULATION']) else "regulates"
            graph.create(Relationship(last_node, regulation, organism_node))
        # Store the last node for each AID
        aid = row['AID']
        if aid not in last_nodes_dict:
            last_nodes_dict[aid] = []
        last_nodes_dict[aid].append(organism_node)
    else:
        # If there is no Organism node, store the last node
        aid = row['AID']
        if aid not in last_nodes_dict:
            last_nodes_dict[aid] = []
        last_nodes_dict[aid].append(last_node)

# Connect ORGANISM_NAME nodes for the same AID
for aid, last_nodes in last_nodes_dict.items():
    organism_nodes = [node for node in last_nodes if isinstance(node, Node) and node.labels == {'Organism'}]
    if organism_nodes:
        organism_node = organism_nodes[0]
        for node in last_nodes:
            if node != organism_node:
                regulation = "regulates"  # Change relationship type as needed
                graph.create(Relationship(node, regulation, organism_node))

In [ ]:
print('End of the Knowledge_Graph_Generation_Algorithm')

End of the Knowledge_Graph_Generation_Algorithm


In [ ]:
#Cypher Query: Output all built knowledge graphs

'''
MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m
'''

In [ ]:
#Cypher Query: Output all knowledge graphs with AID

'''
MATCH (sid:SID {id: 123092557})-[:regulates]->(aid:AID {id: 513226})-[r*]->(n)
RETURN sid, aid, r, n
'''